In [1]:
%cd ..

/home/psushko/WinningEdge/WinningEdge


In [2]:
from winedge import Card, Evaluator
import pandas as pd
from winedge import adding_rank, straight_risk_eval
from winodds import add_aggression_column

In [3]:
pk = pd.read_pickle('./dataframe/poker_dataframe_w_flush.pkl')

#Subsetting the dataframe to only contain games that made it to the river
df_no_folds = pk[(~pk['River'].isnull()) & (~pk['SB cards'].isnull()) & (~pk['BB cards'].isnull())]

In [4]:
def filter_function(row):
    '''
    This function drops all rows of the dataframe that were checked on the river.
    
    '''
    target_list = [('k', 0), ('k', 0)] 
    return row['River actions'] != target_list

# drop rows that went check check
df_no_checks= df_no_folds[df_no_folds.apply(filter_function, axis=1)].reset_index(drop=True) 

In [5]:
df = df_no_checks.dropna(subset=['River actions']).reset_index(drop=True)

In [6]:
df

,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,Turn,River,SB stack,BB stack,SB cards,BB cards,Folded pre,Board,Flush Count,Flush
0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]","[5c, 9c, 5s]",7h,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]",False,"[5c, 9c, 5s, 7h, 2c]",3.0,1.0
1,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[6s, 7s, 5h]",7c,2d,2.65,2,"[ 5c, Ad ]","[ 6h, 9h ]",False,"[6s, 7s, 5h, 7c, 2d]",2.0,0.0
2,42,502873402,Bottrop-0.01-0.02-USD-NoLimitHoldem-Pacific-6-...,gunchozaurs,scs1914,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[2d, 6h, 5d]",8d,Kc,0.83,2,"[ 7d, 6s ]","[ 2h, 8h ]",False,"[2d, 6h, 5d, 8d, Kc]",3.0,1.0
3,80,603643073,Carmen-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,geordieowens,rustman1987,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.04), (r, 0.08), (c, 0.04)]","[7d, 5d, Kc]",Ad,Jc,1,0.64,"[ Kh, Qd ]","[ 3s, Js ]",False,"[7d, 5d, Kc, Ad, Jc]",3.0,1.0
4,99,502599804,Carson-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,_xFaktor,MAX_Rom,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]","[Jh, 5c, 8d]",2c,3h,1.83,0.95,"[ 6s, 6d ]","[ 3c, 5s ]",False,"[Jh, 5c, 8d, 2c, 3h]",2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30788,520653,528310493,Baia Mare-0.01-0.02-USD-NoLimitHoldem-Pacific-...,1gadfly1,Xuantruong31,"[(r, 0.05), (c, 0.04)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.06), (r, 0.18), (c, 0.12)]","[(b, 0.33), (c, 0.33)]","[2d, 8d, Kc]",7s,Jd,2.40,2.52,"[ Kh, 9d ]","[ Ah, 4d ]",False,"[2d, 8d, Kc, 7s, Jd]",3.0,1.0
30789,520793,553224222,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Reggieowl,silviu2010,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.07), (c, 0.07)]","[3c, 5s, 8d]",Qh,7h,0.80,0.83,"[ 5c, 9h ]","[ 3d, Kc ]",False,"[3c, 5s, 8d, Qh, 7h]",2.0,0.0
30790,520794,553224237,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[Kh, Js, 7d]",3s,6d,0.92,0.70,"[ Jc, 9d ]","[ Td, 7h ]",False,"[Kh, Js, 7d, 3s, 6d]",2.0,0.0
30791,520796,553224255,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(b, 0.04), (c, 0.04)]","[7s, 6s, Js]",3c,Ad,0.82,0.79,"[ 6d, Qs ]","[ Ac, 4h ]",False,"[7s, 6s, Js, 3c, Ad]",3.0,1.0


In [ ]:
result_df = adding_rank.add_rank_to_pkl_df('ml/df_filtered_w_flush.pkl')

In [ ]:
df.to_pickle('ml/df_filtered_w_flush.pkl')

In [7]:
result_df = pd.read_pickle('poker_dataframe_w_rank.pkl')
result_df

,level_0,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,...,SB cards,BB cards,Folded pre,Board,Flush Count,Flush,SB Handrank,BB Handrank,SB Hand Strength,BB Hand Strength
0,0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]",...,"[Qh, Tc]","[6h, 2h]",False,"[5c, 9c, 5s, 7h, 2c]",3.0,1.0,Pair,Two Pair,0.274457,0.559501
1,1,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]",...,"[5c, Ad]","[6h, 9h]",False,"[6s, 7s, 5h, 7c, 2d]",2.0,0.0,Two Pair,Two Pair,0.574913,0.575717
2,2,42,502873402,Bottrop-0.01-0.02-USD-NoLimitHoldem-Pacific-6-...,gunchozaurs,scs1914,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]",...,"[7d, 6s]","[2h, 8h]",False,"[2d, 6h, 5d, 8d, Kc]",3.0,1.0,Pair,Two Pair,0.307022,0.577727
3,3,80,603643073,Carmen-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,geordieowens,rustman1987,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.04), (r, 0.08), (c, 0.04)]",...,"[Kh, Qd]","[3s, Js]",False,"[7d, 5d, Kc, Ad, Jc]",3.0,1.0,Pair,Pair,0.524792,0.465291
4,4,99,502599804,Carson-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,_xFaktor,MAX_Rom,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]",...,"[6s, 6d]","[3c, 5s]",False,"[Jh, 5c, 8d, 2c, 3h]",2.0,0.0,Pair,Two Pair,0.298311,0.561244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30788,30788,520653,528310493,Baia Mare-0.01-0.02-USD-NoLimitHoldem-Pacific-...,1gadfly1,Xuantruong31,"[(r, 0.05), (c, 0.04)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.06), (r, 0.18), (c, 0.12)]","[(b, 0.33), (c, 0.33)]",...,"[Kh, 9d]","[Ah, 4d]",False,"[2d, 8d, Kc, 7s, Jd]",3.0,1.0,Pair,High Card,0.510319,0.163093
30789,30789,520793,553224222,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Reggieowl,silviu2010,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.07), (c, 0.07)]",...,"[5c, 9h]","[3d, Kc]",False,"[3c, 5s, 8d, Qh, 7h]",2.0,0.0,Pair,Pair,0.273519,0.222192
30790,30790,520794,553224237,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]",...,"[Jc, 9d]","[Td, 7h]",False,"[Kh, Js, 7d, 3s, 6d]",2.0,0.0,Pair,Pair,0.456044,0.339319
30791,30791,520796,553224255,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(b, 0.04), (c, 0.04)]",...,"[6d, Qs]","[Ac, 4h]",False,"[7s, 6s, Js, 3c, Ad]",3.0,1.0,Pair,Pair,0.317073,0.538060


In [8]:
result_df['Aggr preflop'] = result_df['Preflop actions'].apply(add_aggression_column.check_aggression)
result_df['Aggr flop'] = result_df['Flop actions'].apply(add_aggression_column.check_aggression)
result_df['Aggr turn'] = result_df['Turn actions'].apply(add_aggression_column.check_aggression)
result_df['Aggr river'] = result_df['River actions'].apply(add_aggression_column.check_aggression)

In [9]:
result_df

,level_0,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,...,Flush Count,Flush,SB Handrank,BB Handrank,SB Hand Strength,BB Hand Strength,Aggr preflop,Aggr flop,Aggr turn,Aggr river
0,0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]",...,3.0,1.0,Pair,Two Pair,0.274457,0.559501,"(0, 0)","(0, 0)","(0, 0)","(1, 0)"
1,1,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]",...,2.0,0.0,Two Pair,Two Pair,0.574913,0.575717,"(1, 0)","(0, 0)","(0, 0)","(1, 0)"
2,2,42,502873402,Bottrop-0.01-0.02-USD-NoLimitHoldem-Pacific-6-...,gunchozaurs,scs1914,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]",...,3.0,1.0,Pair,Two Pair,0.307022,0.577727,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
3,3,80,603643073,Carmen-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,geordieowens,rustman1987,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.04), (r, 0.08), (c, 0.04)]",...,3.0,1.0,Pair,Pair,0.524792,0.465291,"(0, 0)","(0, 0)","(1, 0)","(1, 1)"
4,4,99,502599804,Carson-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,_xFaktor,MAX_Rom,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]",...,2.0,0.0,Pair,Two Pair,0.298311,0.561244,"(0, 0)","(0, 0)","(1, 0)","(1, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30788,30788,520653,528310493,Baia Mare-0.01-0.02-USD-NoLimitHoldem-Pacific-...,1gadfly1,Xuantruong31,"[(r, 0.05), (c, 0.04)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.06), (r, 0.18), (c, 0.12)]","[(b, 0.33), (c, 0.33)]",...,3.0,1.0,Pair,High Card,0.510319,0.163093,"(1, 0)","(0, 0)","(1, 1)","(1, 0)"
30789,30789,520793,553224222,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Reggieowl,silviu2010,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.07), (c, 0.07)]",...,2.0,0.0,Pair,Pair,0.273519,0.222192,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30790,30790,520794,553224237,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]",...,2.0,0.0,Pair,Pair,0.456044,0.339319,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30791,30791,520796,553224255,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Reggieowl,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(b, 0.04), (c, 0.04)]",...,3.0,1.0,Pair,Pair,0.317073,0.538060,"(0, 0)","(0, 0)","(0, 1)","(1, 0)"


In [12]:
#dropping all unnecessary columns
df = df.drop('Player SB', axis=1)
df = df.drop('Player BB', axis=1)
df = df.drop('Flop', axis=1)
df = df.drop('Turn', axis=1)
df = df.drop('River', axis=1)
df = df.drop('index', axis=1)
df = df.drop('Flush Count', axis=1)
df = df.drop('level_0', axis=1)
df = df.drop('Folded pre', axis=1)

df = df.drop('Preflop actions', axis=1)
df = df.drop('Flop actions', axis=1)
df = df.drop('Turn actions', axis=1)
df = df.drop('River actions', axis=1)

KeyError: "['index_0'] not found in axis"

In [10]:
df=result_df

In [16]:
df

,level_0,Game ID,File,SB stack,BB stack,SB cards,BB cards,Folded pre,Board,Flush,SB Handrank,BB Handrank,SB Hand Strength,BB Hand Strength,Aggr preflop,Aggr flop,Aggr turn,Aggr river
0,0,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,2,1,"[Qh, Tc]","[6h, 2h]",False,"[5c, 9c, 5s, 7h, 2c]",1.0,Pair,Two Pair,0.274457,0.559501,"(0, 0)","(0, 0)","(0, 0)","(1, 0)"
1,1,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,2.65,2,"[5c, Ad]","[6h, 9h]",False,"[6s, 7s, 5h, 7c, 2d]",0.0,Two Pair,Two Pair,0.574913,0.575717,"(1, 0)","(0, 0)","(0, 0)","(1, 0)"
2,2,502873402,Bottrop-0.01-0.02-USD-NoLimitHoldem-Pacific-6-...,0.83,2,"[7d, 6s]","[2h, 8h]",False,"[2d, 6h, 5d, 8d, Kc]",1.0,Pair,Two Pair,0.307022,0.577727,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
3,3,603643073,Carmen-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,1,0.64,"[Kh, Qd]","[3s, Js]",False,"[7d, 5d, Kc, Ad, Jc]",1.0,Pair,Pair,0.524792,0.465291,"(0, 0)","(0, 0)","(1, 0)","(1, 1)"
4,4,502599804,Carson-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,1.83,0.95,"[6s, 6d]","[3c, 5s]",False,"[Jh, 5c, 8d, 2c, 3h]",0.0,Pair,Two Pair,0.298311,0.561244,"(0, 0)","(0, 0)","(1, 0)","(1, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30788,30788,528310493,Baia Mare-0.01-0.02-USD-NoLimitHoldem-Pacific-...,2.40,2.52,"[Kh, 9d]","[Ah, 4d]",False,"[2d, 8d, Kc, 7s, Jd]",1.0,Pair,High Card,0.510319,0.163093,"(1, 0)","(0, 0)","(1, 1)","(1, 0)"
30789,30789,553224222,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.80,0.83,"[5c, 9h]","[3d, Kc]",False,"[3c, 5s, 8d, Qh, 7h]",0.0,Pair,Pair,0.273519,0.222192,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30790,30790,553224237,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.92,0.70,"[Jc, 9d]","[Td, 7h]",False,"[Kh, Js, 7d, 3s, 6d]",0.0,Pair,Pair,0.456044,0.339319,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30791,30791,553224255,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.82,0.79,"[6d, Qs]","[Ac, 4h]",False,"[7s, 6s, Js, 3c, Ad]",1.0,Pair,Pair,0.317073,0.538060,"(0, 0)","(0, 0)","(0, 1)","(1, 0)"


In [18]:
df

,Game ID,File,SB stack,BB stack,SB cards,BB cards,Board,Flush,SB Handrank,BB Handrank,SB Hand Strength,BB Hand Strength,Aggr preflop,Aggr flop,Aggr turn,Aggr river
0,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,2,1,"[Qh, Tc]","[6h, 2h]","[5c, 9c, 5s, 7h, 2c]",1.0,Pair,Two Pair,0.274457,0.559501,"(0, 0)","(0, 0)","(0, 0)","(1, 0)"
1,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,2.65,2,"[5c, Ad]","[6h, 9h]","[6s, 7s, 5h, 7c, 2d]",0.0,Two Pair,Two Pair,0.574913,0.575717,"(1, 0)","(0, 0)","(0, 0)","(1, 0)"
2,502873402,Bottrop-0.01-0.02-USD-NoLimitHoldem-Pacific-6-...,0.83,2,"[7d, 6s]","[2h, 8h]","[2d, 6h, 5d, 8d, Kc]",1.0,Pair,Two Pair,0.307022,0.577727,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
3,603643073,Carmen-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,1,0.64,"[Kh, Qd]","[3s, Js]","[7d, 5d, Kc, Ad, Jc]",1.0,Pair,Pair,0.524792,0.465291,"(0, 0)","(0, 0)","(1, 0)","(1, 1)"
4,502599804,Carson-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,1.83,0.95,"[6s, 6d]","[3c, 5s]","[Jh, 5c, 8d, 2c, 3h]",0.0,Pair,Two Pair,0.298311,0.561244,"(0, 0)","(0, 0)","(1, 0)","(1, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30788,528310493,Baia Mare-0.01-0.02-USD-NoLimitHoldem-Pacific-...,2.40,2.52,"[Kh, 9d]","[Ah, 4d]","[2d, 8d, Kc, 7s, Jd]",1.0,Pair,High Card,0.510319,0.163093,"(1, 0)","(0, 0)","(1, 1)","(1, 0)"
30789,553224222,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.80,0.83,"[5c, 9h]","[3d, Kc]","[3c, 5s, 8d, Qh, 7h]",0.0,Pair,Pair,0.273519,0.222192,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30790,553224237,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.92,0.70,"[Jc, 9d]","[Td, 7h]","[Kh, Js, 7d, 3s, 6d]",0.0,Pair,Pair,0.456044,0.339319,"(0, 0)","(1, 0)","(1, 0)","(1, 0)"
30791,553224255,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,0.82,0.79,"[6d, Qs]","[Ac, 4h]","[7s, 6s, Js, 3c, Ad]",1.0,Pair,Pair,0.317073,0.538060,"(0, 0)","(0, 0)","(0, 1)","(1, 0)"


In [19]:
df.to_pickle('df_filtered_w_flush_strength_aggr.pkl')